In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pycoingecko import CoinGeckoAPI

In [11]:
cg = CoinGeckoAPI()
print(cg.get_price(ids='bitcoin', vs_currencies='usd'))

{'bitcoin': {'usd': 36051}}
